# Baseline Classification

Identifying the 10, 25, 50, 100 most common plant species. Submit them as a baseline classification strategy.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from plantclef.spark import get_spark

spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/10 14:48:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/10 14:48:16 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [ ]:
import os
from pathlib import Path

# Get list of stored filed in cloud bucket
root = Path(os.path.expanduser("~"))
! date

Mon Feb 10 02:48:23 PM EST 2025


In [ ]:
# Path and dataset names
data_path = f"{root}/p-dsgt_clef2025-0/shared/plantclef/data/parquet_files/"

# Define the path to the train and test parquet files
train_path = f"{data_path}/train"

# Read the parquet files into a spark DataFrame
train_df = spark.read.parquet(train_path)

# Show the data
train_df.printSchema()
train_df.show(n=5)

root
 |-- image_name: string (nullable = true)
 |-- path: string (nullable = true)
 |-- data: binary (nullable = true)
 |-- organ: string (nullable = true)
 |-- species_id: integer (nullable = true)
 |-- obs_id: long (nullable = true)
 |-- license: string (nullable = true)
 |-- partner: string (nullable = true)
 |-- author: string (nullable = true)
 |-- altitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- gbif_species_id: string (nullable = true)
 |-- species: string (nullable = true)
 |-- genus: string (nullable = true)
 |-- family: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- references: string (nullable = true)
 |-- url: string (nullable = true)
 |-- learn_tag: string (nullable = true)
 |-- image_backup_url: string (nullable = true)



+--------------------+--------------------+--------------------+-----+----------+----------+--------------------+-------+----------------+--------+-----------------+------------------+---------------+--------------------+-------------+----------+--------+-----------+--------------------+--------------------+---------+--------------------+
|          image_name|                path|                data|organ|species_id|    obs_id|             license|partner|          author|altitude|         latitude|         longitude|gbif_species_id|             species|        genus|    family| dataset|  publisher|          references|                 url|learn_tag|    image_backup_url|
+--------------------+--------------------+--------------------+-----+----------+----------+--------------------+-------+----------------+--------+-----------------+------------------+---------------+--------------------+-------------+----------+--------+-----------+--------------------+--------------------+---------

In [ ]:
from pyspark.sql import functions as F

# Get top species by number of images
grouped_train_df = (
    train_df.groupBy(["species", "species_id"])
    .agg(F.count("species_id").alias("n"))
    .orderBy(F.col("n").desc())
)

# Action
grouped_train_df.show()

+--------------------+----------+---+
|             species|species_id|  n|
+--------------------+----------+---+
|Styphnolobium jap...|   1369068|823|
|Frangula alnus Mill.|   1360257|793|
|Aria edulis (Will...|   1737559|770|
|Lathyrus oleraceu...|   1741625|700|
|  Bromus sterilis L.|   1414366|674|
|   Lotus hirsutus L.|   1743246|668|
|Scandosorbus inte...|   1737669|662|
|Oxalis dillenii J...|   1394359|653|
|     Hedera helix L.|   1363575|653|
|Calicotome spinos...|   1359160|642|
|Castanea sativa M...|   1358610|638|
|Buxus semperviren...|   1356729|636|
|    Arbutus unedo L.|   1358500|634|
|Pistacia lentiscu...|   1356290|632|
|   Ulmus minor Mill.|   1360607|630|
|Tecomaria capensi...|   1400151|627|
|     Quercus ilex L.|   1357416|626|
|Calamagrostis are...|   1722433|625|
|Pittosporum tobir...|   1394624|625|
|Cercis siliquastr...|   1359162|624|
+--------------------+----------+---+
only showing top 20 rows

